In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['./cnn_finetune'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

tensorflow 1.13.1
keras 2.2.4


In [3]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [4]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from cnn_finetune.resnet_50 import resnet50_model

In [5]:
# Example to fine-tune on 3000 samples from Cifar10
model_name = 'resnet_50'
dataset_name = 'cifar10'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 10
batch_size = 64
epochs = 100
load_weights = True
fine_tuning = True

# Load Cifar10 data. Please implement your own load_data() module for your own dataset
if dataset_name == 'cifar10':
    X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)
elif dataset_name == 'cifar100':
    X_train, Y_train, X_valid, Y_valid = load_cifar100_data(img_rows, img_cols)
else:
    raise('no method for loading dataset')

cifar10 trainset: <class 'numpy.ndarray'> (50000, 32, 32, 3) <class 'numpy.ndarray'> (50000, 1)
cifar10 valset: <class 'numpy.ndarray'> (10000, 32, 32, 3) <class 'numpy.ndarray'> (10000, 1)
----------------------------------------------------------------------------------------------------


In [6]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

datagen.fit(X_train)

train_gen = datagen.flow(X_train, Y_train, batch_size=batch_size)

print(len(train_gen))

782


In [7]:
# Load our model
model = resnet50_model(img_rows, img_cols, channel, num_classes, fine_tuning, load_weights)

Instructions for updating:
Colocations handled automatically by placer.
resnet50_model: classes: 10 fine-tuning: True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]              

In [8]:
# Learning rate is changed to 0.001
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

checkpoint = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name +"-e" + "{epoch:02d}-{val_loss:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             period=1)

checkpoint_best = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name + "-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=False,
                                  mode='auto', 
                                  period=1)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

tensorBoard = TensorBoard(log_dir='./models/' + model_name +'/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

csvLogger = CSVLogger(filename='./models/'+ model_name + '/' + dataset_name + "-log.csv", separator=',', append=False)

# Start Fine-tuning
# model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           shuffle=True,
#           verbose=1,
#           validation_data=(X_valid, Y_valid),
#           callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
#           )

# model.fit_generator(Train_image_generator_Iterator,
#           steps_per_epoch = steps_per_epoch,
#           epochs=epochs,
#           shuffle=True,
#           verbose=1,
#           validation_data=Val_image_generator_Iterator,
#           validation_steps=validation_steps,
#           workers=50,
#           use_multiprocessing=True,
#           callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
#           )

model.fit_generator(train_gen,
          epochs=epochs,
          steps_per_epoch = len(train_gen),
          verbose=1,
          validation_data=(X_valid, Y_valid),
          workers=10,
          use_multiprocessing=True,
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
782/782 [==============================] - 251s 321ms/step - loss: 0.5729 - acc: 0.8673 - val_loss: 2.0167 - val_acc: 0.4192

Epoch 00001: val_loss improved from inf to 2.01673, saving model to ./models/resnet_50/cifar10-e01-2.0167.hd5

Epoch 00001: val_loss improved from inf to 2.01673, saving model to ./models/resnet_50/cifar10-best.hd5
Epoch 2/100
 79/782 [==>...........................] - ETA: 3:14 - loss: 0.2724 - acc: 0.9353

Process ForkPoolWorker-5:
Process ForkPoolWorker-9:
Process ForkPoolWorker-3:
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call las

KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/numpy_array_iterator.py", line 153, in _get_batches_of_transformed_samples
    x.astype(self.dtype), params)
  File "/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/image_data_generator.py", line 853, in apply_transform
    order=self.interpolation_order)
  File "/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/affine_transformations.py", line 330, in apply_affine_transform
    cval=cval) for x_channel in x]
  File "/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/affine_transformations.py", line 330, in <listcomp>
    cval=cval) for x_channel in x]
KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/scipy/ndimage/interpolation.py", line 480, in affine_transform
    output, order, mode, cval, None, None)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
print(Y_valid.shape, predictions_valid.shape)

In [ ]:
# import numpy as np
# np.set_printoptions(threshold=np.inf)
# print(Y_valid)
# print(predictions_valid)

In [ ]:
# print('cross-entropy:', sklearn.metrics.log_loss(Y_valid, predictions_valid))

# y_true = np.argmax(Y_valid, axis=1)
# y_hat = np.argmax(predictions_valid, axis=1)
# # print('report:', sklearn.metrics.classification_report(y_true, y_hat))
# print('acc:', sklearn.metrics.accuracy_score(y_true, y_hat))
# # print('roc:', sklearn.metrics.roc_auc_score(y_true, y_hat))
# # print('confusion_matrix:\n', sklearn.metrics.confusion_matrix(y_true, y_hat))